# 📒 **Daily Journal Analyzer: A GenAI-Powered Mental Health Assistant** 🚀

---

## 📝 **Introduction**

In today’s fast-paced world, journaling has emerged as a simple yet powerful tool to reflect on our thoughts, manage stress, and monitor emotional well-being. However, it’s often difficult for individuals to objectively interpret their own writing or derive actionable insights from it.

This notebook introduces a lightweight **Generative AI-powered Daily Journal Analyzer**, built using **Google’s Gemini API**. With just a short paragraph of text describing how your day went, the assistant provides a structured emotional analysis including:
- Mood classification
- Emotional tone and confidence
- Key thought patterns
- Mental health suggestions

Unlike traditional machine learning solutions, this project does **not require any dataset, training, or labeling**. Instead, it leverages **few-shot prompting**, **structured output (JSON mode)**, and the **Gemini LLM** to extract meaningful insights from unstructured text in real time.

This tool is intended for students, working professionals, or anyone looking for a personal, AI-powered mental wellness assistant — without sacrificing privacy or needing advanced technical setup.

---

## 🎯 **Objective**

The primary goal of this project is to develop an AI-powered assistant that can analyze free-form journal entries and provide structured mental health insights.

Specifically, this system will:

- Accept **natural language journal entries** as input  
- Use **few-shot prompting** to guide Gemini’s understanding of emotions  
- Return a **structured JSON response** containing:
  - Overall **mood classification** (e.g., positive, negative, neutral)
  - Detected **emotional tone** (e.g., stressed, content, anxious)
  - Detected **confidence score** and **stress level** (e.g., high, low)
  - Brief analysis of the user's **self awareness**
  - **Important words** related to mood or triggers
  - **Key thought patterns** extracted from the entry
  - **Actionable suggestions** to improve mental well-being in the analysis

The tool demonstrates how **Generative AI** can be used as a supportive mental wellness assistant, even with zero external data and minimal coding effort — making it accessible and practical for real-world use.

---

## 🧠 **GenAI Capabilities Used**

This project demonstrates the practical use of multiple Generative AI capabilities, integrated seamlessly using Google’s Gemini API:

| ✅ Capability                     | 💡 How It’s Used                                                                                                       |
|-----------------------------------|------------------------------------------------------------------------------------------------------------------------|
| **Structured Output (JSON Mode)** | The AI returns a consistent, machine-readable JSON response with fields like mood, tone, suggestions, etc.             |
| **Few-shot Prompting**            | Carefully crafted prompt includes example journal entries and expected JSON-format responses.                          |
| **Function Calling (Simulated)**  | The model acts like a callable function: input journal → output structured insights, without defining custom functions.|
| **Long Context Window**           | Gemini can process long journal entries while preserving coherence in the analysis.                                    |

⚠️ Note: While **Embeddings**, **Vector Search**, or **RAG** are not used in this project, the architecture can be extended to include them in future iterations (e.g., tracking mood trends over time).

---

## ⚙️ **Environment Setup**
In this project I'll going to use google's gemeni api key. Before using the Gemini API, I need to install and import the required packages.

In [ ]:
# Install the Google Generative AI SDK and Scikit-learn
!pip install -q -U google-generativeai
!pip install -q scikit-learn

In [ ]:
# Import necessary libraries
import google.generativeai as genai
import os
import warnings
warnings.filterwarnings("ignore")

---

## 🔐 **API Key Setup (Using Kaggle Secrets)**

To securely access the Gemini API, I’ll store the API key using Kaggle Secrets. Follow these steps before running the code:

### ✅ **Step 1: Add Your API Key to Kaggle Secrets**
1. Go to your Kaggle profile → "Account" → scroll to "API" section.
2. If you haven’t already, get your [Google API key](https://aistudio.google.com/app/apikey).
3. In your Kaggle notebook, click on the **"Add-ons"** → **"Secrets"**.
4. Create a new secret with:
   - **Key**: `GOOGLE_API_KEY`
   - **Value**: *Your actual Gemini API Key*

### ✅ **Step 2: Load and Configure the API Key**

In [ ]:
from kaggle_secrets import UserSecretsClient

# Access the Gemini API key securely
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")

# Configure the SDK
import google.generativeai as genai
genai.configure(api_key=api_key)

---

## 📜 **Prompt Design**

The prompt is designed using **few-shot prompting** to guide the Gemini model in extracting structured emotional insights from unstructured text. It clearly explains the task and provides examples to demonstrate the expected response format.

We instruct the model to:

1. Read the **journal entry** carefully
2. Identify the **user’s mood**, **emotional tone**, and **stress level**
3. Summarize **key thoughts**
4. Suggest simple, helpful **mental health tips**
5. Return output as **JSON**

### 💭 **Prompt Template**
This design ensures Gemini understands both **format** and **intent**, reducing hallucination and keeping output structured.

In [ ]:
prompt_template = """
You are a helpful AI assistant that analyzes emotional journal entries and returns a structured JSON output including the following keys: 
- mood
- emotional_tone
- confidence (0.0 to 1.0)
- stress_level (Low, Medium, High)
- self_reflection (Brief analysis of the user's self-awareness)
- keywords (Important words related to mood or triggers)
- key_thoughts
- suggestions

Respond only with a valid JSON object. Do not add explanations or extra text.

Here are a few examples:

---

Journal: "Today was amazing. I finished my assignment early and went out with friends. Felt really good after a long time."

Response:
{
  "mood": "positive",
  "emotional_tone": "relieved",
  "confidence": 0.92,
  "stress_level": "Low",
  "self_reflection": "The user is self-aware of their need for both productivity and social time.",
  "keywords": ["assignment", "friends", "relaxed"],
  "key_thoughts": "User feels productive and socially fulfilled.",
  "suggestions": "Keep up the momentum by maintaining balance between work and relaxation."
}

---

Journal: "I couldn’t focus at all. Everything feels off, and I’m getting anxious about the upcoming exams."

Response:
{
  "mood": "negative",
  "emotional_tone": "anxious",
  "confidence": 0.87,
  "stress_level": "High",
  "self_reflection": "User recognizes their struggle with focus and anxiety, indicating self-awareness.",
  "keywords": ["focus", "anxious", "exams"],
  "key_thoughts": "User is feeling overwhelmed and stressed about academics.",
  "suggestions": "Take short breaks, practice deep breathing, and focus on one task at a time."
}

---

Now analyze this journal entry:
\"\"\"{user_input}\"\"\"
"""

---

## 🤖 **Model Initialization & Prompt Execution**

Now that I’ve crafted my prompt, let’s initialize the Gemini model and define a function that takes a journal entry as input and returns structured insights in JSON format.

### 🔧 **Model Initialization**
I am going to use google's ```gemini-2.0-flash``` model for this project.

In [ ]:
# Load the Gemini Pro model
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

### 🗒️ **Journal Analyzer & Save Journal Function**
The ```analyze_journal_entry``` function analyze the Journal using ```gemini-2.0-flash``` model and the ```save_journal_entry``` funcrion tracks record of entries with timestamp.

In [ ]:
import re
import json
import datetime
from sklearn.feature_extraction.text import CountVectorizer

# This function will help to extacct keyword from the entry
def extract_keywords(text, top_n=3):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform([text])
    keywords = vectorizer.get_feature_names_out()
    return keywords[:top_n].tolist()

# List to store journal entries and their analysis results
journal_history = []

def analyze_journal_entry(entry: str):
    """Takes a journal entry and returns structured mental health insights."""
    full_prompt = prompt_template.replace("{user_input}", entry)
    response = model.generate_content(full_prompt)
    text = response.text.strip()

    try:
        json_match = re.search(r"\{[\s\S]*\}", text)
        if json_match:
            parsed_output = json.loads(json_match.group())

            # Inject keywords if not already present
            if "keywords" not in parsed_output or not parsed_output["keywords"]:
                parsed_output["keywords"] = extract_keywords(entry)

            return parsed_output
        else:
            raise ValueError("No JSON found in model output.")
    except Exception as e:
        print("Error parsing model output:", e)
        print("Raw response:", text)
        return {"error": "Invalid response format"}

def save_journal_entry(entry, analysis_result):
    """Save journal entry with analysis results and timestamp."""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    entry_data = {
        "timestamp": timestamp,
        "entry": entry,
        "mood": analysis_result.get("mood"),
        "emotional_tone": analysis_result.get("emotional_tone"),
        "stress_level": analysis_result.get("stress_level"),
        "keywords": analysis_result.get("keywords"),
        "confidence": analysis_result.get("confidence")
    }
    journal_history.append(entry_data)


### 🧮 **Mood Trend Calculation & Plotting the Mood Trend**

This function will calculate the trend based on mood values and visualize the trend of mood over time. I'll use `1` for positive, `0` for neutral, and `-1` for negative.

In [ ]:
import matplotlib.pyplot as plt

def calculate_mood_trend():
    """Calculate the mood trend (Improving/Worsening) based on past entries."""
    mood_values = []

    # Extract mood values: 1 for positive, 0 for neutral, -1 for negative
    for entry in journal_history:
        if entry["mood"] == "positive":
            mood_values.append(1)
        elif entry["mood"] == "negative":
            mood_values.append(-1)
        else:
            mood_values.append(0)

    # Calculate the trend from the last 5 entries (you can adjust this number)
    if len(mood_values) > 1:
        trend = "Improving" if sum(mood_values[-5:]) > 0 else "Worsening"
    else:
        trend = "No data"
    
    return trend

def plot_mood_trend():
    """Plot mood trend over time."""
    mood_values = []
    timestamps = []

    # Collect mood values and timestamps
    for entry in journal_history:
        if entry["mood"] == "positive":
            mood_values.append(1)
        elif entry["mood"] == "negative":
            mood_values.append(-1)
        else:
            mood_values.append(0)
        timestamps.append(entry["timestamp"])

    # Plot the mood trend
    plt.figure(figsize=(10, 5))
    plt.plot(timestamps, mood_values, marker='o', color='b', label="Mood Trend")
    plt.xlabel('Timestamp')
    plt.ylabel('Mood (Positive=1, Neutral=0, Negative=-1)')
    plt.title('Mood Trend Over Time')
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.grid(True)
    plt.show()

def display_mood_trend():
    """Display the mood trend (Improving/Worsening) and plot."""
    trend = calculate_mood_trend()
    print(f"> Mood Trend: {trend}")
    plot_mood_trend()  # Display the plot

### 🧩 **Example Run**
This is an example to test the model. You can rerun this with any journal entry and receive real-time, structured insights.

In [ ]:
# Example journal entry
entry = "Today I felt overwhelmed with all my college work, but I managed to finish a few tasks."

# Analyze the journal entry (using your existing function)
analysis_result = analyze_journal_entry(entry)

# Save the journal entry with analysis result
save_journal_entry(entry, analysis_result)

print(json.dumps(analysis_result, indent=2))

---

## ✍️ **Try It Yourself: Analyze Your Own Journal Entry**
This section lets you try your own journal entry. By default, it runs with a sample entry so the notebook remains fully executable without user input. Try it to get better mood trend result.

In [ ]:
# Optional: Let the user try their own journal entry
def try_custom_entry(enable_input=False):
    if enable_input:
        try:
            user_input = input("✍️ Enter your journal entry: ")
        except EOFError:
            user_input = ""  # Handles non-interactive kernel runs
    else:
        # Use a default test string if not in interactive mode
        user_input = "I don't feel motivated lately. Even small tasks feel overwhelming."

    if user_input.strip():
        result = analyze_journal_entry(user_input)
        save_journal_entry(user_input, result)
        print(json.dumps(result, indent=2))
    else:
        print("⚠️ Skipped: No input provided.")

# Set True to enable user_input
try_custom_entry(enable_input=False)

---

## 🔃 **Sample Journal Texts (Optional)**
This code block contains some sample day-wise logs which will append to the ```journal_history``` with randomized ```hour``` and ```minute``` for each day's timestamp. Uncomment this block if you want ```mood trend graph``` becomes meaningful even without user input.

💡 **Tip:** Uncomment the lines below to load sample journal entries with simulated daily timestamps.

In [ ]:
## Optional: Uncomment this block to analyze and store sample journal entries
## This simulates daily entries with random times for better graph visualization


# from datetime import datetime, timedelta
# import random

# sample_journal_texts = [
#     "Had a productive morning but felt overwhelmed by college deadlines.",
#     "Went out with friends after a long time. Felt good and relaxed.",
#     "Couldn’t sleep well last night. Mind was restless and I kept overthinking.",
#     "Completed most of my tasks today. Feeling accomplished and calm.",
#     "Felt anxious and irritable all day for no clear reason.",
#     "Spent the afternoon reading and enjoying the quiet. It helped me feel centered."
# ]

## Start from today and go backward one day per entry
# base_date = datetime.now()

# for i, text in enumerate(sample_journal_texts):
#     result = analyze_journal_entry(text)
#     if "error" not in result:
#         # Simulate random hour and minute
#         random_hour = random.randint(6, 22)   # picks a random hour between 6am to 10 pm
#         random_minute = random.randint(0, 59)
#         simulated_datetime = (base_date - timedelta(days=i)).replace(
#             hour=random_hour, minute=random_minute, second=0, microsecond=0
#         )
#         timestamp = simulated_datetime.strftime("%Y-%m-%d %H:%M:%S")

#         entry_data = {
#             "timestamp": timestamp,
#             "entry": text,
#             "mood": result.get("mood"),
#             "emotional_tone": result.get("emotional_tone"),
#             "stress_level": result.get("stress_level"),
#             "keywords": result.get("keywords"),
#             "confidence": result.get("confidence")
#         }
#         journal_history.append(entry_data)

---

## 📈 **Visualize Mood Trend Results**
With the help of ```matplotlib``` library let's visualize the mood trends over the time:

In [ ]:
# Display mood trend (Improving/Worsening) and plot
print("\n🧠 AI-Powered Mental Health Analysis:")
display_mood_trend()

---

## 📑 **Analysis & Limitations**

### 📊 **Analysis**

- The model successfully translates unstructured journal entries into structured insights (like mood classification, stress indicators, and self-esteem reflections).
- Outputs follow a consistent JSON format, making them ideal for further analysis, storage, or integration into dashboards.
- The prompt design ensures that responses remain contextually relevant, emotionally aware, and psychologically interpretable.

### 🟡 **Limitations**

1. **Lack of Clinical Accuracy**  
   - This tool is not a substitute for professional mental health diagnosis or therapy.
   - It lacks clinical validation and can miss nuanced or critical psychological symptoms.

2. **Bias in Responses**  
   - Large Language Models may reflect biases present in training data.
   - This can result in insensitive or culturally inappropriate interpretations.

3. **Inconsistent Output Formatting**  
   - Despite JSON-mode prompts, the model occasionally returns responses that are not valid JSON.
   - Error handling has been implemented, but edge cases may still occur.

4. **No Personalization or History Awareness**  
   - Each journal entry is evaluated in isolation — there's no memory of past inputs or longitudinal tracking.
   - This limits deeper trend analysis across time.

5. **Data Privacy and Security**  
   - Although no user data is stored in this notebook, deploying such tools in real-world settings would require strict compliance with privacy standards like HIPAA or GDPR.

6. **Prompt Sensitivity**  
   - Slight variations in journal entry phrasing can yield different outputs.
   - Prompt engineering improvements can reduce inconsistency, but not eliminate it.

---

⚠️ **Ethical Note**  
This tool is built for **educational purposes only**. Do not use it to make mental health decisions. Encourage users to consult certified professionals when in need.

---


## 📑 **Conclusion & Future Work**

### ✅ **Conclusion**

In this notebook, we built a beginner-friendly, Generative AI-powered tool to analyze personal journal entries and extract structured mental health insights. By combining:

- **Prompt engineering**
- **Structured JSON generation**
- **Google's Gemini API**

…we were able to turn subjective narratives into machine-readable summaries that categorize emotional states, detect stress indicators, and provide supportive affirmations.

The notebook runs **completely end-to-end** without requiring dataset creation, making it a practical demo of Generative AI's real-world applications — especially in the field of **emotional wellbeing and self-reflection**.


### 🚀 **Future Work**

While this project demonstrates the potential of Generative AI for emotional journaling and mental health insights, there are several avenues for meaningful expansion and improvement:

1. **Emotion Timeline and Sentiment Progression**  
   - Introduce fine-grained **emotion progression analysis** across longer entries or over multiple days. This would help users visualize emotional shifts within a single entry or over time — offering a clearer picture of mental well-being.

2. **Secure Cloud Storage and User Authentication**  
   - Integrate user-level access and secure storage via Firebase or Supabase. Each user’s journal history would be isolated and encrypted — enabling long-term tracking and privacy protection.

3. **Daily Reflection Summary Generator**  
   - At the end of each day or week, the AI could auto-generate a **summary paragraph** based on all entries, highlighting emotional patterns, key concerns, and positive moments — acting as a virtual therapist's note.

4. **Multimodal & Multilingual Input**  
   - Allow voice inputs (audio understanding) or image-based journaling (e.g., photos, drawings) for broader expression modes and also enable the AI to understand and analyze journal entries in **regional languages** or mixed-language inputs, especially useful in diverse linguistic contexts like India.

5. **Evaluation of AI Responses**  
   - Integrate **Generative AI Evaluation** to assess the quality, consistency, and helpfulness of AI feedback across journal entries using automated metrics or feedback loops from users.
   
6. **Mental Health Resource Recommendation System**  
   - Based on analysis, the system can **recommend mental health content**, exercises, or coping strategies using RAG (Retrieval Augmented Generation), improving practical utility and user outcomes.

6. **Mobile App or Chatbot Deployment**  
   - Package the system into a frontend interface using Gradio or Streamlit, or deploy as a mobile journaling companion.

---

⭐ **This project** showcases the potential of Generative AI to provide structured, personalized mental health insights from everyday reflections. With continued development, it aspires to become a supportive companion in emotional self-care and digital well-being.

---